In [7]:
import pandas as pd
import feather

from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical
from keras.models import load_model


In [8]:
n_groups_per_col = 10
architecture = "2_dense_512"
path = "../data/models/dep/categorical/" + architecture +"/"

data_train = pd.read_feather(path + "data_train.feather").to_numpy()
labels_train = pd.read_feather(path + "labels_train.feather").to_numpy()

In [9]:
def fit_ind_categorical_model(data_train, labels_train):
    
    model = Sequential()
    model.add(Dense(units=512, activation='relu', input_shape=(data_train.shape[1],)))
    model.add(Dense(units=512, activation='relu'))
    model.add(Dense(pow(n_groups_per_col, 3), activation='softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    history = model.fit(data_train, labels_train, validation_split=0.2, batch_size=512, epochs=5, verbose=1)
    
    return (history, model)

In [10]:
(history, model) = fit_ind_categorical_model(data_train, labels_train)
feather.write_dataframe(pd.DataFrame.from_dict(low_history.history), path + "history.feather", compression="uncompressed")
low_model.save(path + "model.hdf5")


Train on 4342137 samples, validate on 1085535 samples
Epoch 1/20
2145792/4342137 [=============>................] - ETA: 2:40 - loss: 6.9333 - accuracy: 0.0020

KeyboardInterrupt: 

In [9]:
### Predict
# model = load_model(path + "model.hdf5")


In [10]:
data_test = pd.read_feather(path + "data_test.feather").to_numpy()

pred_prob = model.predict_proba(data_test, batch_size = 512)
feather.write_dataframe(pd.DataFrame(pred_prob), path + "pred_prob.feather", compression='uncompressed')
